In [1]:
# Install package for Lasso
if(!require("glmnet")){
    install.packages("glmnet")
}

# Install package for Pliable Lasso
if(!require("pliable")){
    url <- "https://cran.r-project.org/src/contrib/Archive/pliable/pliable_1.1.tar.gz"
    pkgFile <- "pliable_1.1.tar.gz"
    download.file(url = url, destfile = pkgFile)
    install.packages(pkgs=pkgFile, type="source", repos=NULL)
    unlink(pkgFile)
}

Loading required package: glmnet

Loading required package: Matrix

Loaded glmnet 3.0-2


Loading required package: pliable

Loading required package: class



## Lasso vs Pliable Lasso

In [2]:
library(pliable)
n <- 100; p <- 10; nz <- 4
x <- scale(matrix(rnorm(n*p), n, p))
z <- scale(matrix(sample(c(0, 1), n * nz, replace=TRUE), n, nz))
y <- 2 * x[,1] - 2 * x[,2] + 2 * x[,3] + 2 * z[,1] * x[,3] + 2 * x[,4] - 4 * z[,2] * x[,4] + 0.5 * rnorm(n)
xz <- matrix(0, n, p*nz) # interaction terms
for (i in 1:p) {
    for (j in 1:nz) {
        xz[,nz*(i-1)+j] <- x[,i] * z[,j]
    }
}
x_xz <- cbind(x, xz)
fit_glmnet <- glmnet(x_xz, y)
cv_fit_glmnet <- cv.glmnet(x_xz, y)
fit_glmnet <- glmnet(x_xz, y, lambda=cv_fit_glmnet$lambda.min)
beta <- fit_glmnet$beta[1:p]
theta <- t(matrix(fit_glmnet$beta[(p+1):(p*(nz+1))], nz, p))
print('beta')
print(beta)
print('theta')
print(theta)
# [1] "beta"
#  [1]  1.93037241 -1.99080091  1.90628910  1.87340843 -0.01680067  0.00000000
#  [7]  0.00000000 -0.02436891  0.00000000  0.00000000
# [1] "theta"
#             [,1]     [,2]       [,3]        [,4]
#  [1,] 0.07002739  0.00000 0.00000000 -0.01425233
#  [2,] 0.00000000  0.00000 0.00000000  0.00000000
#  [3,] 1.98184381  0.00000 0.00000000  0.00000000
#  [4,] 0.00000000 -3.99627 0.00000000  0.00000000
#  [5,] 0.00000000  0.00000 0.06648736 -0.09642435
#  [6,] 0.00000000  0.00000 0.00000000  0.00000000
#  [7,] 0.00000000  0.00000 0.00000000  0.00000000
#  [8,] 0.00000000  0.00000 0.02567855  0.00000000
#  [9,] 0.02011390  0.00000 0.00000000  0.00000000
# [10,] 0.00000000  0.00000 0.00000000  0.0000000

[1] "beta"
 [1]  1.918005230 -1.998049657  1.856284381  1.854920406  0.036556885
 [6]  0.000000000  0.000000000  0.000000000  0.003684479  0.000000000
[1] "theta"
             [,1]         [,2]         [,3]       [,4]
 [1,] -0.01829025  0.003070454  0.000000000 0.00000000
 [2,] -0.03237145  0.000000000 -0.024000678 0.00000000
 [3,]  1.95362420  0.000000000  0.000000000 0.00000000
 [4,]  0.00000000 -3.894796769  0.046323525 0.04941705
 [5,]  0.00000000  0.001118701  0.003434282 0.00000000
 [6,]  0.00000000  0.000000000  0.000000000 0.00000000
 [7,]  0.01223750  0.055096062  0.000000000 0.00000000
 [8,]  0.00000000  0.000000000  0.000000000 0.00000000
 [9,]  0.00000000  0.032768797  0.000000000 0.00000000
[10,]  0.00000000  0.000000000  0.000000000 0.00000000


In [3]:
fit_pliable <- pliable(x, z, y)
cv_fit_pliable <- cv.pliable(fit_pliable, x, z, y, verbose=FALSE)
fit_pliable <- pliable(x, z, y, lambda=cv_fit_pliable$lambda.min)
print(coef(fit_pliable))
# $betaz
# [1]  0.01100791 -0.06641263  0.10949380  0.03993577
# 
# $beta
#  [1]  1.96552143 -2.01533988  1.95736209  1.88119097 -0.08082027 -0.03948929
#  [7]  0.00000000 -0.07799262 -0.02051081 -0.02314566
# 
# $theta
#               [,1]         [,2]         [,3]         [,4]
#  [1,]  0.087065566 -0.000131529  0.000000000 -0.041261783
#  [2,]  0.000000000  0.000000000  0.000000000  0.000000000
#  [3,]  1.941422377 -0.034798144  0.000000000 -0.002797550
#  [4,] -0.005704073 -3.955412349  0.000000000  0.026515585
#  [5,] -0.031601711  0.000000000  0.069946806 -0.076491350
#  [6,] -0.007395559  0.000000000  0.000000000  0.004435835
#  [7,]  0.000000000  0.000000000  0.000000000  0.000000000
#  [8,]  0.000000000 -0.031844665  0.058509824  0.000000000
#  [9,]  0.009148531  0.000000000 -0.006090111 -0.001607628
# [10,]  0.000000000  0.016534991 -0.006355646  0.000000000

$betaz
[1]  0.02404686  0.04942682  0.03209978 -0.02187951

$beta
 [1]  1.905484334 -1.997997005  1.873942956  1.891179723  0.045343292
 [6] -0.031644768  0.005380973 -0.025796615  0.035434040  0.000000000

$theta
              [,1]         [,2]          [,3]          [,4]
 [1,] -0.027843063  0.001157446 -0.0007576882  0.0000000000
 [2,] -0.033248039  0.008083006 -0.0534410129  0.0053225031
 [3,]  1.916532277  0.012557619  0.0230284023 -0.0294272027
 [4,] -0.040074893 -3.835838003  0.0597499103  0.0562799924
 [5,]  0.000000000  0.000000000  0.0000000000  0.0000000000
 [6,]  0.000000000  0.000000000  0.0000000000  0.0000000000
 [7,]  0.012390649  0.022579615  0.0035575108 -0.0013181137
 [8,]  0.000000000  0.000000000  0.0000000000  0.0000000000
 [9,]  0.003499283  0.018288252  0.0000000000  0.0008936914
[10,]  0.000000000  0.000000000  0.0000000000  0.0000000000



In [4]:
mse <- 0
for (k in 1:20) {
    x <- scale(matrix(rnorm(n*p), n, p))
    z <- scale(matrix(sample(c(0, 1), n * nz, replace=TRUE), n, nz))
    y <- 2 * x[,1] - 2 * x[,2] + 2 * x[,3] + 2 * z[,1] * x[,3] + 2 * x[,4] - 4 * z[,2] * x[,4] + 0.5 * rnorm(n)
    xz <- matrix(0, n, p*nz) # interaction terms
    for (i in 1:p) {
        for (j in 1:nz) {
            xz[,nz*(i-1)+j] <- x[,i] * z[,j]
        }
    }
    x_xz <- cbind(x, xz)
    est_y <- predict(fit_glmnet, newx=x_xz, lambda=cv_fit_glmnet$lambda.min)
    mse <- mse + sum((abs(y - est_y))^2)
}
mse <- mse / 20
print(mse)
# [1] 29.47268

[1] 32.09263


In [5]:
mse <- 0
for (k in 1:20) {
    x <- scale(matrix(rnorm(n*p), n, p))
    z <- scale(matrix(sample(c(0, 1), n * nz, replace=TRUE), n, nz))
    y <- 2 * x[,1] - 2 * x[,2] + 2 * x[,3] + 2 * z[,1] * x[,3] + 2 * x[,4] - 4 * z[,2] * x[,4] + 0.5 * rnorm(n)
    est_y <- predict(fit_pliable, x, z, lambda=cv_fit_glmnet$lambda.min)
    mse <- mse + sum((abs(y - est_y))^2)
}
mse <- mse / 20
print(mse)
# [1] 32.92485

[1] 34.19266
